In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/Combined_News_DJIA.csv")

In [3]:
df.head(1)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Adjusted Label,Nasdaq,S&P500
0,8/8/2008,0,Georgia 'downs two Russian warplanes' as count...,BREAKING: Musharraf to be impeached.',Russia Today: Columns of troops roll into Sout...,Russian tanks are moving towards the capital o...,"Afghan children raped with 'impunity,' U.N. of...",150 Russian tanks have entered South Ossetia w...,"Breaking: Georgia invades South Ossetia, Russi...",The 'enemy combatent' trials are nothing but a...,...,This is a busy day: The European Union has ap...,"Georgia will withdraw 1,000 soldiers from Iraq...",Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis: Georgia invades South Osse...,Indian shoe manufactory - And again in a seri...,Visitors Suffering from Mental Illnesses Banne...,"No Help for Mexico's Kidnapping Surge""",1,1,1


In [4]:
df["combined"] = df["Top1"].astype(str)+' '+df['Top2']+' '+df['Top3']+' '+df['Top4']+' '+df['Top5']+' '+df['Top6']+' '+df['Top7']+' '+df['Top8']+' '+df['Top9']+' '+df['Top10']+' '+df['Top11']+' '+df['Top12']+' '+df['Top13']+' '+df['Top14']+' '+df['Top15']+' '+df['Top16']+' '+df['Top17']+' '+df['Top18']+' '+df['Top19']+' '+df['Top20']+' '+df['Top21']+' '+df['Top22']+' '+df['Top23']+' '+df['Top24']+' '+df['Top25']

In [5]:
df.drop([ 'Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7',
       'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15',
       'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23',
       'Top24', 'Top25'],inplace=True,axis=1)

In [6]:
df.head(1)

,Date,Label,Adjusted Label,Nasdaq,S&P500,combined
0,8/8/2008,0,1,1,1,Georgia 'downs two Russian warplanes' as count...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

In [10]:
from sklearn.naive_bayes import MultinomialNB

In [12]:
from sklearn.linear_model import SGDClassifier

In [20]:
import numpy as np

In [15]:
df['combined'] = df['combined'].astype(str)

In [16]:
kaggleTraining = []
kaggleTest = []
for row in range(0,1391):  
    kaggleTraining.append((df["combined"][row].lower()))

for row in range(1392,1988):
    kaggleTest.append((df["combined"][row].lower()))

In [25]:
from sklearn.pipeline import Pipeline

In [27]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [34]:
text_clf = text_clf.fit(kaggleTraining, df["Label"][0:1391])
predicted = text_clf.predict(kaggleTest)
np.mean(predicted == df["Label"][1392:1988])

0.5302013422818792

In [35]:
from sklearn.linear_model import SGDClassifier

In [39]:
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words="english")),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)),
])

In [44]:
_ = text_clf_svm.fit(kaggleTraining, df["Nasdaq"][0:1391])

C:\Users\AK\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [45]:
predicted_svm = text_clf_svm.predict(kaggleTest)
np.mean(predicted_svm == df["Nasdaq"][1392:1988])

0.5318791946308725

In [48]:
import nltk
# nltk.download()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                      ('tfidf', TfidfTransformer()),
                      ('mnb', MultinomialNB(fit_prior=False)),
])



In [49]:
text_mnb_stemmed = text_mnb_stemmed.fit(kaggleTraining,df["Label"][0:1391])
predicted_mnb_stemmed = text_mnb_stemmed.predict(kaggleTest)
np.mean(predicted_mnb_stemmed == df["Label"][1392:1988])

0.5302013422818792